## Imports

In [1]:
# DO NOT CHANGE THESE LINES.
import os
import json
import pandas as pd
import warnings
from sklearn.linear_model import LogisticRegression
from feature_engine.encoding import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from joblib import dump
warnings.filterwarnings('ignore')

## Paths

In [2]:
# DO NOT CHANGE THESE LINES.
ROOT_DIR = os.path.dirname(os.getcwd())
MODEL_INPUTS_OUTPUTS = os.path.join(ROOT_DIR, 'model_inputs_outputs/')
INPUT_DIR = os.path.join(MODEL_INPUTS_OUTPUTS, "inputs")
INPUT_SCHEMA_DIR = os.path.join(INPUT_DIR, "schema")
DATA_DIR = os.path.join(INPUT_DIR, "data")
TRAIN_DIR = os.path.join(DATA_DIR, "training")
TEST_DIR = os.path.join(DATA_DIR, "testing")
MODEL_PATH = os.path.join(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH = os.path.join(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE = os.path.join(MODEL_ARTIFACTS_PATH, 'ohe.joblib')
LABEL_ENCODER_FILE = os.path.join(MODEL_ARTIFACTS_PATH, 'label_encoder.joblib')
PREDICTOR_DIR_PATH = os.path.join(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH = os.path.join(PREDICTOR_DIR_PATH, "predictor.joblib")
IMPUTATION_FILE = os.path.join(MODEL_ARTIFACTS_PATH, 'imputation.joblib')
if not os.path.exists(MODEL_ARTIFACTS_PATH):
    os.makedirs(MODEL_ARTIFACTS_PATH)
if not os.path.exists(PREDICTOR_DIR_PATH):
    os.makedirs(PREDICTOR_DIR_PATH)

### Reading the schema
The schema contains metadata about the datasets. We will use the schema to get information about the type of each feature (NUMERIC or CATEGORICAL) and the id and target features, this will be helpful in preprocessing stage.

In [3]:
file_name = [f for f in os.listdir(INPUT_SCHEMA_DIR) if f.endswith('json')][0]
schema_path = os.path.join(INPUT_SCHEMA_DIR, file_name)
with open(schema_path, "r", encoding="utf-8") as file:
    schema = json.load(file)
features = schema['features']

numeric_features = []
categorical_features = []
for f in features:
    if f['dataType'] == 'CATEGORICAL':
        categorical_features.append(f['name'])
    else:
        numeric_features.append(f['name'])

id_feature = schema['id']['name']
target_feature = schema['target']['name']

### Reading training data

In [4]:
file_name = [f for f in os.listdir(TRAIN_DIR) if f.endswith('.csv')][0]
file_path = os.path.join(TRAIN_DIR, file_name)
df = pd.read_csv(file_path)
df.head()

,u_id,fatals,a_ct,a_ped_f,a_pedal_f,a_roll,a_hr,a_polpur,month,day,...,owner,deaths,numoccs,impact1,deformed,ve_forms,ve_total,weather,lgt_cond,driver_factor
0,32083,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,10,2,...,Driver (in this crash) Was Registered Owner,1,1.0,Clockpoint 12,Disabling damage,1,1,Clear,Dark - not lighted,other
1,55073,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,6,21,...,Driver (in this crash) Not Registered Owner (o...,1,1.0,Clockpoint 1,Disabling damage,1,1,Clear,Daylight,speeding_driver_involved
2,7458,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,7,14,...,Driver (in this crash) Was Registered Owner,0,1.0,Clockpoint 12,NaN,1,1,Clear,Daylight,other
3,5685,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,9,15,...,Driver (in this crash) Not Registered Owner (o...,0,1.0,Clockpoint 12,Functional damage,1,1,Clear,Dark - not lighted,other
4,9245,1,Single-Vehicle Crash,Other Crash,Other Crash,Other Crash,No - Hit and Run,Other Crash,9,28,...,Driver (in this crash) Was Registered Owner,1,1.0,Clockpoint 9,Disabling damage,1,1,Clear,Dark - not lighted,drunk_driver_involved


## Data Preprocessing
Data preprocessing is very important before training the model, as the data may contain missing values in some cells. Moreover, most of the learning algorithms cannot work with categorical data, thus the data has to be encoded.

In this section we will impute the missing values and encode the categorical features. Afterwards the data will be ready to train the model.

##### Imputing missing data
> In this section we imputed the missing values using the mode. However in many cases the mode is not an optimal value. You can try your own imputation and compare the results.

In [5]:
# Imputing missing data

# columns_with_missing_values = df.columns[df.isna().any()]
# imputation_values = {}
# for column in columns_with_missing_values:
#     mode = df[column].mode()[0]
#     df[column].fillna(mode, inplace=True)
#     imputation_values[column] = mode

# path = dump(imputation_values, IMPUTATION_FILE)

# Comment the above code and write you own imputation code here
# Replace missing values with mode or median
columns_with_missing_values = df.columns[df.isna().any()]
imputation_values = {}
for column in columns_with_missing_values:
    if df[column].dtype == 'object':
        mode = df[column].mode()[0]
        df[column].fillna(mode, inplace=True)
        imputation_values[column] = mode
    else:
        median = df[column].median()
        df[column].fillna(median, inplace=True)
        imputation_values[column] = median

path = dump(imputation_values, IMPUTATION_FILE)

##### Encoding Categorical features
> Notice that we do not want to encode the target feature nor the id column.

In [6]:
# Saving the id and target columns in a different variable.
ids = df[id_feature]
target = df[target_feature]

# Dropping the id and target from the dataframe
df.drop(columns=[id_feature, target_feature], inplace=True)

# Ensure that all categorical columns are stored as str type.
# This is to ensure that even if the categories are numbers (1, 2, ...), they still get encoded.
for c in categorical_features:
    df[c] = df[c].astype(str)

# Encoding the features
encoder = OneHotEncoder(top_categories=6)
encoder.fit(df)
df = encoder.transform(df)

# Saving the encoder to use it on the testing dataset
path = dump(encoder, OHE_ENCODER_FILE)


#### Encoding the target feature

In [7]:
encoder = LabelEncoder()
y = encoder.fit_transform(target.values.reshape(-1, 1))
dump(encoder, LABEL_ENCODER_FILE)
y

array([1, 2, 1, ..., 1, 1, 0])

In [8]:
df.head()

,fatals,month,day,hour,minute,age,permvit,pernotmvit,mod_year,deaths,...,weather_Rain,"weather_Fog, smog, smoke",weather_Snow,weather_Reported as unknown,lgt_cond_Daylight,lgt_cond_Dark - not lighted,lgt_cond_Dark - lighted,lgt_cond_Dawn,lgt_cond_Dark - unknown lighting,lgt_cond_Dusk
0,1,10,2,3.0,10.0,62,1,0,2003.0,1,...,0,0,0,0,0,1,0,0,0,0
1,1,6,21,8.0,45.0,40,1,0,2002.0,1,...,0,0,0,0,1,0,0,0,0,0
2,1,7,14,21.0,45.0,26,1,1,2003.0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,9,15,20.0,46.0,64,1,1,1999.0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,9,28,20.0,24.0,45,1,0,1996.0,1,...,0,0,0,0,0,1,0,0,0,0


### Training the Classifier
We choose Logistic Regression Classifier, but feel free to try your own and compare the results.

In [ ]:

####

#from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Creating an XGBoost model and training it
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                   max_depth=1, random_state=0)
model.fit(df, y)

####

# Saving the model to use it for predictions
path = dump(model, PREDICTOR_FILE_PATH)
